In [17]:
import pandas as pd
from tqdm.notebook import tqdm
import sqlite3

In [18]:
chunks = pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)
test_chunk = next(chunks)
test_chunk.loc[(test_chunk['transaction_count'] >= 50) & (test_chunk['average_day_wait'] < 50)]

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
7,1508085911,1730166125,58,67,23.925,43.923
10,1508167040,1730166125,51,51,28.196,52.876
15,1508863549,1730166125,340,391,18.302,42.422
18,1508867870,1730166125,50,79,12.658,26.402
25,1508011040,1730166224,132,145,8.579,28.053
...,...,...,...,...,...,...
9951,1508804113,1730476490,97,102,32.882,54.294
9976,1508834862,1730476979,53,60,34.983,54.899
9979,1508118555,1730476995,84,91,22.791,38.622
9982,1508131087,1730477225,96,323,3.613,21.439


In [19]:
db = sqlite3.connect('../data/hop_teaming_database.sqlite')

for chunk in tqdm(pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)):
    # filter the required conditions (trying to eliminate accidental referrals)
    chunk = chunk.loc[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait'] < 50)] 
    # Append the chunk to a calls table
    # chunk.to_sql('hop_teaming', db, if_exists = 'append', index = False)

0it [00:00, ?it/s]

In [20]:
db.execute('CREATE INDEX from_to_npi ON hop_teaming(from_npi, to_npi)')

In [21]:
db.close()

In [22]:
pd.set_option('display.max_columns', None)

In [23]:
chunks = pd.read_csv('../data/npidata_pfile_20050523-20230212.csv', chunksize = 10000)
test_chunk = next(chunks)

C:\Users\garre\AppData\Local\Temp\ipykernel_10796\3841821350.py:2: DtypeWarning: Columns (17,24,64,68,71,72,73,74,75,76,77,78,79,80,81,82,83,85,86,87,89,90,147,151,155,157,159,161,163,165,167,169,170,171,174,175,178,314) have mixed types. Specify dtype option on import or set low_memory=False.
  test_chunk = next(chunks)


In [24]:
test_chunk.loc[(test_chunk['Healthcare Provider Primary Taxonomy Switch_1'] != 'Y') & (test_chunk['Healthcare Provider Primary Taxonomy Switch_1'] != 'N')]

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Business Mailing Address Telephone Number,Provider Business Mailing Address Fax Number,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Country Code (If outside U.S.),Provider Business Practice Location Address Telephone Number,Provider Business Practice Location Address Fax Number,Provider Enumeration Date,Last Update Date,NPI Deactivation Reason Code,NPI Deactivation Date,NPI Reactivation Date,Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Healthcare Provider Taxonomy Code_1,Provider License Number_1,Provider License Number State Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Provider License Number_2,Provider License Number State Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Provider License Number_3,Provider License Number State Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Provider License Number_4,Provider License Number State Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Provider License Number_5,Provider License Number State Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Provider License Number_6,Provider License Number State Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Provider License Number_7,Provider License Number State Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Provider License Number_8,Provider License Number State Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Provider License Number_9,Provider License Number State Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Provider License Number_10,Provider License Number State Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Provider License Number_11,Provider License Number State Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Provider License Number_12,Provider License Number State Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Provider License Number_13,Provider License Number State Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Provider License Number_14,Provider License Number State Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Provider License Number_15,Provider License Number State Code_15,Healthcare Provider Primary Taxonomy Switch_15,Other Provider Identifier_1,Other Provider Identifier Type Code_1,Other Provider Ide

In [25]:
for i in range(1,16):
    print(test_chunk.loc[test_chunk[f'Healthcare Provider Primary Taxonomy Switch_{i}'] == 'Y', f'Healthcare Provider Taxonomy Code_{i}'].count())

9001
62
2
0
1
0
0
0
0
0
0
0
0
0
0


In [26]:
def find_taxonomy(col):
    for i in range(1, 16):
        taxonomy_switch = f'Healthcare Provider Primary Taxonomy Switch_{i}'
        taxonomy_value = f'Healthcare Provider Taxonomy Code_{i}'
        if col.get(taxonomy_switch) == 'Y':
            return col.get(taxonomy_value)
    return 'no primary taxonomy'

In [27]:
%%capture [--no-stderr]
# the capture above is here so that it doesn't show warnings about columns types and so that I avoid manually setting dozens of columns dtypes!

db = sqlite3.connect('../data/hop_teaming_database.sqlite')

for chunk in tqdm(pd.read_csv('../data/npidata_pfile_20050523-20230212.csv', chunksize = 10000, dtype={'Provider Business Practice Location Address Postal Code': object})):

    # first extract the primary taxonomy
    chunk['Primary Taxonomy'] = chunk.apply(lambda col: find_taxonomy(col), axis=1)

    # Take only first 5 digits from postal code column
    chunk['Provider Business Practice Location Address Postal Code'] = chunk['Provider Business Practice Location Address Postal Code'].str[:5]

    # next, only keep columns we're interested in and renaming so that there are no ()
    chunk = (
        chunk 
        [['NPI',
        'Entity Type Code',
        'Provider Organization Name (Legal Business Name)',
        'Provider Last Name (Legal Name)',
        'Provider First Name',
        'Provider Middle Name',
        'Provider Name Prefix Text',
        'Provider Name Suffix Text',
        'Provider Credential Text',
        'Provider First Line Business Practice Location Address',
        'Provider Second Line Business Practice Location Address',
        'Provider Business Practice Location Address City Name',
        'Provider Business Practice Location Address State Name',
        'Provider Business Practice Location Address Postal Code',
        'Primary Taxonomy']]
        .rename(columns={'Provider Organization Name (Legal Business Name)': 'Provider Organization Name',
        'Provider Last Name (Legal Name)': 'Provider Last Name',
        'Provider Name Prefix Text': 'Provider Name Prefix',
        'Provider Name Suffix Text': 'Provider Name Suffix',
        'Provider Business Practice Location Address City Name': 'City',
        'Provider Business Practice Location Address State Name': 'State',
        'Provider Business Practice Location Address Postal Code': 'Postal Code'})
    )

    # Then clean up the column names
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]

    # Finally, the chunk to a calls table
    chunk.to_sql('nnpes', db, if_exists = 'append', index = False)

In [28]:
query = """
SELECT *
FROM nnpes
WHERE city = 'NASHVILLE'
AND state = 'TN'
"""
with sqlite3.connect('../data/hop_teaming_database.sqlite') as db: 
    npidata_nashville = pd.read_sql_query(query, db)
npidata_nashville

,npi,entity_type_code,provider_organization_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,city,state,postal_code,primary_taxonomy
0,1922001957,1.0,None,PRESLEY,RICHARD,E,None,None,M.D.,2011 MURPHY AVE,STE 302,NASHVILLE,TN,37203,207V00000X
1,1760485817,1.0,None,MORAN,SAM,HOUSTON,None,None,M.D.,329 21ST AVE N,STE 4,NASHVILLE,TN,37203,207V00000X
2,1154324192,1.0,None,OLDFIELD,ELIZABETH,L,None,None,M.D.,2011 MURPHY AVE,STE 200,NASHVILLE,TN,37203,207V00000X
3,1770586786,1.0,None,RICHARDS,SHERRIE,ANDERSON,None,None,M.D.,2201 MURPHY AVE,STE 410,NASHVILLE,TN,37203,207V00000X
4,1629071832,1.0,None,ADKINS,ROYCE,T,None,None,M.D.,2011 MURPHY AVE,STE 200,NASHVILLE,TN,37203,207V00000X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22569,1245690585,1.0,None,HARRISON,ASHLEY,None,MS.,None,None,1005 DR. D.B. TODD JR. BLVD,None,NASHVILLE,TN,37208,122300000X
22570,1710638523,2.0,"VORI MEDICAL NY, PLLC",None,None,None,None,None,None,100 POWELL PL # 1441,None,NASHVILLE,TN,37204,208100000X
22571,1053753541,1.0,None,LESUEUR,JESSICA,J,DR.,None,OD,3443 DICKERSON PIKE,STE. 240,NASHVILLE,TN,37207,152W00000X
22572,1922039346,1.0,None,KOSTAMAA,HEIKKI,E,DR.,None,M.D.,28 WHITE BRIDGE PIKE,STE.208,NASHVILLE,TN,37205,207W00000X


In [29]:
db.execute('CREATE INDEX npi ON nnpes(npi)')

In [30]:
db.close()

In [31]:
db = sqlite3.connect('../data/hop_teaming_database.sqlite')

nucc_taxonomy = pd.read_csv('../data/nucc_taxonomy_230.csv', encoding = 'unicode_escape')

# lowercase column names and replace spaces
nucc_taxonomy.columns = [x.lower().replace(' ', '_') for x in nucc_taxonomy.columns]

# add table to database
nucc_taxonomy.to_sql('nucc_taxonomy', db, if_exists = 'append', index = False)

#create index
db.execute('CREATE INDEX code ON nucc_taxonomy(code)')

db.close()

nucc_taxonomy

,code,grouping,classification,specialization,definition,notes,display_name,section
0,193200000X,Group,Multi-Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,NaN,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,"A physician who specializes in the diagnosis, ...",Source: National Uniform Claim Committee,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,An allergy and immunology physician who specia...,"Source: National Uniform Claim Committee, 2022...",Clinical & Laboratory Immunology (Allergy & Im...,Individual
...,...,...,...,...,...,...,...,...
868,343800000X,Transportation Services,Secured Medical Transport (VAN),NaN,A public or privately owned transportation ser...,NaN,Secured Medical Transport (VAN),Non-Individual
869,344600000X,Transportation Services,Taxi,NaN,A land commercial vehicle used for the transpo...,NaN,Taxi,Non-Individual
870,347D00000X,Transportation Services,Train,NaN,An organization or business licensed to provid...,NaN,Train,Non-Individual
871,347E00000X,Transportation Services,Transportation Broker,NaN,An organization that provides transportation f...,Source: Section 6083 of the Deficit Reduction ...,Transportation Broker,Non-Individual


In [32]:
db = sqlite3.connect('../data/hop_teaming_database.sqlite')

zip_cbsa = pd.read_excel('../data/ZIP_CBSA_122021.xlsx', index_col = None, header = 0, dtype={'zip': object})

# add table to database
zip_cbsa.to_sql('zip_cbsa', db, if_exists = 'append', index = False)

#create index
db.execute('CREATE INDEX zip ON zip_cbsa(zip)')

db.close()

zip_cbsa

,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,00683,41900,SAN GERMAN,PR,0.999842,1.0,1.0,0.999855
1,00683,32420,SAN GERMAN,PR,0.000158,0.0,0.0,0.000145
2,00923,41980,SAN JUAN,PR,1.000000,1.0,1.0,1.000000
3,01010,44140,BRIMFIELD,MA,0.976896,1.0,1.0,0.977816
4,01010,49340,BRIMFIELD,MA,0.023104,0.0,0.0,0.022184
...,...,...,...,...,...,...,...,...
47479,60684,16980,CHICAGO,IL,0.000000,1.0,0.0,1.000000
47480,33945,15980,PINELAND,FL,0.000000,0.0,1.0,1.000000
47481,78144,99999,PANNA MARIA,TX,0.000000,1.0,0.0,1.000000
47482,12257,10580,ALBANY,NY,0.000000,1.0,0.0,1.000000
